<a href="https://colab.research.google.com/github/tejaspradhan/Face-Generation-from-Textual-Features-using-GANs/blob/main/Stacked_GAN_T2F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
sys.path.insert(0, '/content/stylegan2')
import pretrained_networks
from dnnlib import tflib as tflib
import numpy as np
import PIL.Image
import dnnlib
from matplotlib.pyplot import imshow
%matplotlib inline

## Custom Methods

In [ ]:
!git clone https://github.com/NVlabs/stylegan2.git

In [114]:
def resize_image(img):
  mywidth = 1024
  wpercent = (mywidth/float(img.size[0]))
  hsize = int((float(img.size[1])*float(wpercent)))
  img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
  return img

In [87]:
def stylegan_generate_image(latent_vector):
  %tensorflow_version 1.x
  network_pkl="gdrive:networks/stylegan2-ffhq-config-f.pkl"
  truncation_psi=1.0
  seed=np.random.randint(10000)
  print('Loading networks from "%s"...' % network_pkl)
  _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
  print('---- Loaded ----')
  noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
  Gs_kwargs = dnnlib.EasyDict()
  Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
  Gs_kwargs.randomize_noise = False
  if truncation_psi is not None:
      Gs_kwargs.truncation_psi = truncation_psi

  print('Generating image for seed %d...' % (seed))
  rnd = np.random.RandomState(seed)
  z = np.array([latent_vector])
  print("Noise Vars: ",noise_vars)
  print("Input size: ",*Gs.input_shape[1:])
  # print("Input: \n",latent_vector)
  tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) 
  images = Gs.run(z, None, **Gs_kwargs)
  img=PIL.Image.fromarray(images[0], 'RGB')
  return img 

## Loading Data

In [19]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [20]:
! kaggle datasets download -d jessicali9530/celeba-dataset

100% 1.32G/1.33G [00:43<00:00, 31.7MB/s]
100% 1.33G/1.33G [00:43<00:00, 33.0MB/s]


In [ ]:
!unzip "/content/celeba-dataset.zip" -d "/content/data"

In [119]:
embeddings = np.load('/content/embeddings_37k.npz')['arr_0']

In [120]:
embeddings.shape

(37370, 512)

In [121]:
face_data = pd.read_csv('/content/face_features_preprocessed.csv')

In [122]:
face_ids=face_data['image_id']

## Training 

In [ ]:
#generated_vector = model.predict(embedding)
#gen_image = stylegan_generate_image(generated_vector)
# loss = loss(gen_image,real_images[ith])
